In [19]:
from gpkit import Model, Variable, units
import gpkit
from numpy import pi, tan
import math
from __future__ import division
gpkit.settings["latex_modelname"] = False

class ltaHALE(Model):
    def setup(self):

        constraints = []
        
        solar = False

        # Constants
        g = Variable('g', 9.81,'m/s^2', 'gravitational constant')
        R_earth = Variable('R_{earth}', 6371*10^3, 'meters', 'radius of the earth')
        R = Variable('R', 287, 'J/kg*K', 'ideal gas constant')

        # Requirements
        m_pay = Variable('m_{pay}', 10, 'lb', 'mass of payload') 
        Inclination_Angle = Variable('angle_{incl}', 5, 'deg', 'max antenna inclination angle')
        time = Variable('t', 5, 'days', 'flight time')
        P_payload = Variable('P_{pay}', 10, 'W', 'power requirement of payload')

        # Assumptions
        Cd = Variable('C_d', 0.2, '-', 'drag coefficient') #Sphere
        eta_prop = Variable(r'\eta_{prop}', 0.75, '-', 'propulsive efficiency')
        m_bat = Variable('m_{bat}', 2, 'kg', 'mass of payload battery')
        m_avi = Variable('m_{avi}', 1, 'kg', 'mass of avionics')
        m_fixed = Variable('m_{fixed}', 'lbm', 'fixed mass')

        constraints.extend([m_fixed >= m_pay + m_avi + m_bat])

        W = Variable('W', 'lbf', 'vehicle weight')
        rho = Variable(r'\rho','kg/m^3', 'air density')
        h = Variable("h", "km", "Altitude")
        vel = Variable('vel', 'm/s', 'cruise velocity')
        v_wind = Variable('v_{wind}', 'm/s', 'wind velocity')
        conversion = Variable('conversion',835.29,'m/s*km**(1.409)','wind velocity coversion to m/s')

        constraints.extend([vel >= v_wind,
                           v_wind>=conversion*h**(-1.409)])


#        # LTA Sizing
        t = Variable('thickness ratio', 0.5, '-', 'thickness ratio')
        c = Variable('chord', 'm', 'length of vehicle')
        area_front = Variable('area_{front}', 'm^2', 'frontal area')
        area_surface = Variable('area_{surface}', 'm^2', 'surface area')
        r = Variable('r', 'm', 'vehicle radius')
        vol = Variable('vol', 'm^3', 'vehicle Volume')
        rho_he = Variable(r'\rho_{he}', 'g/L', 'density of helium at altitude')
        R_he = Variable('R_{he}', 2077, 'J/(kg * K)', 'specific gas constant for helium')
        rho_h = Variable(r'\rho_{h}', 'g/L', 'density of hydrogen at altitude')
        R_h = Variable('R_{h}', 4124, 'J/(kg * K)', 'specific gas constant for hydrogen')
        t_skin = Variable('t_{skin}', 0.005, 'in', 'thickness of skin of balloon')
        rho_skin = Variable(r'\rho_{skin}', 1.45, 'g/cm^3', 'density of polyester skin')
        m_balloon = Variable('m_balloon', 'kg', 'mass of balloon skin')
        constraints.extend([c >= (vol/(10 * pi * t * (2/3 * 0.2969 - 0.126 * 0.5 - 0.3516 * 1/3 + 0.2843 * 1/4 - 0.1015 * 1/5)))**(1/3),
                            vol >= W/(rho * g),
                            r == t * c/2,
                            area_front == pi * r**2,
                            area_surface == 4 * pi * r**2,
                            m_balloon == t_skin* area_surface * rho_skin])

        # Power Consumption
        P_shaft = Variable('P_{shaft}', 'W', 'Shaft power')
        E_total = Variable('E_{total}', "MJ", "total mission energy")
        constraints.extend([P_shaft >= 1 /eta_prop/2. * rho * vel**3 * Cd * area_front,
                            E_total >= (P_payload + P_shaft)*time])

        # Carbon Fuels
        h_diesel = Variable('h_{diesel}', 35.86, 'MJ/L', 'diesel energy density')
        W_diesel = Variable('W_{diesel}', 'lbf', 'mission diesel fuel weight')
        Diesel_dens = Variable(r'\Rho_{diesel}', 0.832, 'kg/L', 'diesel density')
        m_engine = Variable('m_engine', 'kg', 'mass of engine')
        PtoW = Variable('PtoW', 109, 'W/kg', 'engine power to weight ratio') # https://en.wikipedia.org/wiki/Power-to-weight_ratio Toyota Venza
        Diesel_eff = Variable(r'\Eta_{diesel}', 0.5, '-', 'diesel engine efficiency')
        constraints.extend([W_diesel >= E_total/h_diesel*Diesel_dens*g,
                            m_engine >= P_shaft / (PtoW * Diesel_eff)])
#         Gasoline_Edens = Variable('gasoline_{edens}', 32.4, 'MJ/L', 'gasoline energy density')
#         Gasoline_eff = Variable(r'\Eta_{gasoline}', 0.25, '-', 'gasoline engine efficiency')
#         Gasoline_dens = Variable(r'\Rho_{gasoline}', 0.75, 'kg/L', 'gasoline density')
#         Jetfuel_Edens = Variable('jetfuel_{edens}', 37.4, 'MJ/L', 'jetfuel energy density')
#         Jetfuel_dens = Variable(r'\Rho_{jetfuel}', 0.81, 'kg/L', 'jetfuel density')
#         constraints.extend([Diesel_needed >= E_total_joules/(Diesel_Edens * 10^6),
#                             Gasoline_needed >= E_total_joules/(Gasoline_Edens * 10^6),
#                             Jetfuel_needed >= E_total_joules/(Jetfuel_Edens * 10^6),
#                             Diesel_mass >= Diesel_needed*Diesel_dens,
#                             Gasoline_mass >= Gasoline_needed * Gasoline_dens,
#                             Jetfuel_mass >= Jetfuel_needed * Jetfuel_dens,
#                             Diesel_fraction >= Diesel_mass/(W/g),
#                             Gasoline_fraction >= Gasoline_mass/(W/g),
#                             Jetfuel_fraction >= Jetfuel_mass/(W/g)])

        # Atmosphere model        
        p_sl = Variable("p_{sl}", 101325, "Pa", "Pressure at sea level")
        T_sl = Variable("T_{sl}", 288.15, "K", "Temperature at sea level")
        L_atm = Variable("L_{atm}", 0.0065, "K/m", "Temperature lapse rate")
        T_atm = Variable("T_{atm}", "K", "air temperature")
        M_atm = Variable("M_{atm}", 0.0289644, "kg/mol", "Molar mass of dry air")
        R_atm = Variable("R_{atm}", 8.31447, "J/mol/K")
        TH = (g*M_atm/R_atm/L_atm).value.magnitude  # dimensionless
        constraints.extend([h <= 20000*units.m,  # Model valid to top of troposphere
                            h >=14900*units.m, #for footprint
                            T_sl >= T_atm + L_atm*h,     # Temp decreases w/ altitude
                            rho <= p_sl*T_atm**(TH-1)*M_atm/R_atm/(T_sl**TH)])  # http://en.wikipedia.org/wiki/Density_of_air#Altitude
        
        P_atm = Variable('P_{atm}', 'Pa', 'air pressure')
        constraints.extend([P_atm == rho * R_atm/M_atm * T_atm,
                            rho_h == P_atm/(R_h * T_atm),
                            rho_he == P_atm/(R_he * T_atm)])
        
        # station keeping requirement
        footprint = Variable("d_{footprint}", 300, 'km', "station keeping footprint diameter")
        lu = Variable(r"\theta_{look-up}", 5, '-', "look up angle")
        R_earth = Variable("R_{earth}", 6371, "km", "Radius of earth")
        tan_lu = lu*pi/180. + (lu*pi/180.)**3/3.  # Taylor series expansion
        # approximate earth curvature penalty as distance^2/(2*Re)
        constraints.extend([h >= tan_lu*0.5*footprint + footprint**2/8./R_earth])
        
        constraints.extend([W >= m_fixed * g + W_diesel + m_balloon * g + vol * rho_he * g+m_engine*g])
        
        objective = W

        return objective, constraints

m = ltaHALE()
m.solve('cvxopt')
#m.controlpanel()


Using solver 'cvxopt'
Solving for 20 variables.
Solving took 0.022 seconds.

Cost
----
 42.74 [lbf] 

Free Variables
--------------
     E_{total} : 69.8      [MJ]      total mission energy           
       P_{atm} : 4324      [Pa]      air pressure                   
     P_{shaft} : 151.6     [W]       Shaft power                    
       T_{atm} : 158.2     [K]       air temperature                
             W : 42.74     [lbf]     vehicle weight                 
    W_{diesel} : 3.571     [lbf]     mission diesel fuel weight     
          \rho : 0.09524   [kg/m**3] air density                    
     \rho_{he} : 0.01316   [g/l]     density of helium at altitude  
      \rho_{h} : 0.006629  [g/l]     density of hydrogen at altitude
  area_{front} : 6.469     [m**2]    frontal area                   
area_{surface} : 25.87     [m**2]    surface area                   
         chord : 5.74      [m]       length of vehicle              
             h : 20        [km]      Alt

{'constants': {\theta_{look-up}_ltaHALE2: array(5),
  d_{footprint}_ltaHALE2: array(300),
  R_{earth}_ltaHALE2: array(6371),
  h_{diesel}_ltaHALE2: array(35.86),
  \Rho_{diesel}_ltaHALE2: array(0.832),
  PtoW_ltaHALE2: array(109),
  \Eta_{diesel}_ltaHALE2: array(0.5),
  R_{he}_ltaHALE2: array(2077),
  R_{h}_ltaHALE2: array(4124),
  t_{skin}_ltaHALE2: array(0.005),
  \rho_{skin}_ltaHALE2: array(1.45),
  T_{sl}_ltaHALE2: array(288.15),
  p_{sl}_ltaHALE2: array(101325),
  L_{atm}_ltaHALE2: array(0.0065),
  M_{atm}_ltaHALE2: array(0.0289644),
  R_{atm}_ltaHALE2: array(8.31447),
  conversion_ltaHALE2: array(835.29),
  thickness ratio_ltaHALE2: array(0.5),
  g_ltaHALE2: array(9.81),
  C_d_ltaHALE2: array(0.2),
  \eta_{prop}_ltaHALE2: array(0.75),
  m_{bat}_ltaHALE2: array(2),
  m_{avi}_ltaHALE2: array(1),
  P_{pay}_ltaHALE2: array(10),
  t_ltaHALE2: array(5),
  m_{pay}_ltaHALE2: array(10)},
 'cost': array(42.73946554194787),
 'freevariables': {W_{diesel}_ltaHALE2: array(3.5714794541183763),
